In [1]:
#loading the dataset

In [2]:
import zipfile
import os

path = '/content/drive/MyDrive/archive (16).zip'
extract_path = '/content/ex_images'
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

os.listdir(extract_path)

['Fruit And Vegetable Diseases Dataset']

In [3]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils import shuffle

In [4]:
# List of fruits and their categories
columns = [
    'Apple__Healthy', 'Apple__Rotten',
    'Banana__Healthy', 'Banana__Rotten',
    'Bellpepper__Healthy', 'Bellpepper__Rotten',
    'Carrot__Healthy', 'Carrot__Rotten',
    'Cucumber__Healthy', 'Cucumber__Rotten',
    'Grape__Healthy', 'Grape__Rotten',
    'Guava__Healthy', 'Guava__Rotten',
    'Jujube__Healthy', 'Jujube__Rotten',
    'Mango__Healthy', 'Mango__Rotten',
    'Orange__Healthy', 'Orange__Rotten',
    'Pomegranate__Healthy', 'Pomegranate__Rotten',
    'Potato__Healthy', 'Potato__Rotten',
    'Strawberry__Healthy', 'Strawberry__Rotten',
    'Tomato__Healthy', 'Tomato__Rotten'
]

In [5]:
# Define the main fruit categories
fruit_categories = ['Apple', 'Banana', 'Bellpepper', 'Carrot', 'Cucumber', 'Grape',
                    'Guava', 'Jujube', 'Mango', 'Orange', 'Pomegranate', 'Potato',
                    'Strawberry', 'Tomato']

In [6]:
# Function to allow user to select a fruit type
def select_fruit_type():
    print("Select a fruit type:")
    for i, fruit in enumerate(fruit_categories, 1):
        print(f"{i}. {fruit}")
    try:
        choice = int(input("Enter the number corresponding to the fruit type: ")) - 1
        if 0 <= choice < len(fruit_categories):
            return fruit_categories[choice]
        else:
            print("Invalid choice. Please enter a valid number.")
            return None
    except ValueError:
        print("Invalid input. Please enter a number.")
        return None


In [7]:
# Function to load images and labels for a selected fruit type
def load_images(base_path, fruit_type):
    healthy_images = []
    rotten_images = []
    img_size = 100

    # Load healthy images
    healthy_path = os.path.join(base_path, f"{fruit_type}__Healthy")
    for img in os.listdir(healthy_path):
        image_path = os.path.join(healthy_path, img)
        image = cv2.imread(image_path)
        if image is not None:
            image = cv2.resize(image, (img_size, img_size))
            image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            image_gray = image_gray.reshape(img_size, img_size, 1)
            healthy_images.append(image_gray)

    # Load rotten images
    rotten_path = os.path.join(base_path, f"{fruit_type}__Rotten")
    for img in os.listdir(rotten_path):
        image_path = os.path.join(rotten_path, img)
        image = cv2.imread(image_path)
        if image is not None:
            image = cv2.resize(image, (img_size, img_size))
            image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            image_gray = image_gray.reshape(img_size, img_size, 1)
            rotten_images.append(image_gray)

    # Create labels
    healthy_labels = [0] * len(healthy_images)  # 0 for healthy
    rotten_labels = [1] * len(rotten_images)    # 1 for rotten

    # Combine images and labels
    images = np.array(healthy_images + rotten_images)
    labels = np.array(healthy_labels + rotten_labels)

    return images, labels


In [8]:
if __name__ == "__main__":
    selected_fruit = select_fruit_type()
    if not selected_fruit:
        exit()

    # Load images and labels for the selected fruit type
    base_path = '/content/ex_images/Fruit And Vegetable Diseases Dataset'
    images, labels = load_images(base_path, selected_fruit)

    # Convert lists to numpy arrays
    X = np.array(images).astype(float) / 255  # Normalize images
    y = np.array(labels)

    # Shuffle the data
    X, y = shuffle(X, y, random_state=42)

    # Splitting data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Reshape images to flatten them into 1D arrays
    X_train_flattened = X_train.reshape(X_train.shape[0], -1)
    X_test_flattened = X_test.reshape(X_test.shape[0], -1)

    # Initialize SVM model
    svm_model = SVC(kernel='linear', random_state=42)

    # Train the SVM model
    svm_model.fit(X_train_flattened, y_train)

    # Predict on the test set
    y_pred = svm_model.predict(X_test_flattened)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {accuracy:.2f}')

    # Generate classification report
    print(classification_report(y_test, y_pred))

    # Allow user to test the model with a custom image
    test_image_path = input("Enter the path to your test image: ")
    if os.path.exists(test_image_path):
        test_image = cv2.imread(test_image_path)
        test_image = cv2.resize(test_image, (100, 100))  # Resize to match training size
        test_image_gray = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)
        test_image_gray = test_image_gray.reshape(100, 100, 1)
        test_image_normalized = test_image_gray.astype(float) / 255

        # Predict using the trained model
        prediction = svm_model.predict(test_image_normalized.flatten().reshape(1, -1))

        # Print the prediction
        if prediction == 0:
            print("Predicted: Healthy")
        elif prediction == 1:
            print("Predicted: Rotten")
        else:
            print("Unknown prediction")
    else:
        print("Invalid image path.")

Select a fruit type:
1. Apple
2. Banana
3. Bellpepper
4. Carrot
5. Cucumber
6. Grape
7. Guava
8. Jujube
9. Mango
10. Orange
11. Pomegranate
12. Potato
13. Strawberry
14. Tomato
Enter the number corresponding to the fruit type: 1
Accuracy: 0.73
              precision    recall  f1-score   support

           0       0.68      0.72      0.70       477
           1       0.77      0.73      0.75       597

    accuracy                           0.73      1074
   macro avg       0.73      0.73      0.73      1074
weighted avg       0.73      0.73      0.73      1074

Enter the path to your test image: 1
Invalid image path.
